# Dùng **fasttext** để train, tạo ra embedding  nhờ đó có thể suggest các bài tin tức liên quan nhất tới cụm từ người dùng input.

In [3]:
import gensim
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from gensim.models.word2vec import LineSentence
import multiprocessing
import pandas as pd
cores = multiprocessing.cpu_count()
print(cores)

8


In [4]:
from regress_adapter import get_news_date

data = get_news_date('2022-01-01', '2023-09-31')
print(data.head())

/Users/ngocp/Documents/projects/pyml/botapp/algorithm/regress_adapter.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


                                               title  \
0  Số liệu kinh tế Mỹ sẽ tác động mạnh tới giá vàng?   
1  Câu chuyện 1/5 lao động trẻ thất nghiệp ở Trun...   
2  Chứng khoán thế giới và giá dầu đồng loạt tăng...   
3  Vàng hồi giá mạnh khi đồng USD quay đầu giảm, ...   
4     Trung Quốc giảm đầu tư vào các nước phương Tây   

                                             content                date  
0  \nGiá vàng thế giới khởi động tuần giao dịch m... 2023-07-31 10:41:00  
1  \nMệt mỏi vì áp lực phải thành công với tư các... 2023-07-31 08:46:00  
2  \nThị trường chứng khoán thế giới và giá dầu c... 2023-07-29 08:41:00  
3  \nGiá vàng thế giới phục hồi khá mạnh trong ph... 2023-07-29 09:33:00  
4  \nMới chỉ vài năm trước, dòng vốn từ Trung Quố... 2023-07-31 08:46:00  


In [5]:
t = data['title'] + data['content']
print(t.head(), type(t))
data_list = t.to_list()

0    Số liệu kinh tế Mỹ sẽ tác động mạnh tới giá và...
1    Câu chuyện 1/5 lao động trẻ thất nghiệp ở Trun...
2    Chứng khoán thế giới và giá dầu đồng loạt tăng...
3    Vàng hồi giá mạnh khi đồng USD quay đầu giảm, ...
4    Trung Quốc giảm đầu tư vào các nước phương Tây...
dtype: object <class 'pandas.core.series.Series'>


In [6]:
import helper
import numpy as np
from importlib import reload

reload(helper)

def clear_text_vi(texts = []):
    clear_ld = lambda t: helper.NLP(t).get_words_feature()
    t = map(clear_ld, texts)
    return list(t)

print(data_list[:2])
data_token = clear_text_vi(data_list)
print(data_token[:5])

['Số liệu kinh tế Mỹ sẽ tác động mạnh tới giá vàng?\nGiá vàng thế giới khởi động tuần giao dịch mới trong xu thế giảm nhẹ, kéo giá vàng trong nước giảm theo. Giới phân tích cho rằng loạt số liệu quan trọng của kinh tế Mỹ công bố trong tuần này có thể ảnh hưởng nhiều đến triển vọng lãi suất của Cục Dự trữ Liên bang Mỹ (Fed), từ đó tác động không nhỏ đến diễn biến giá vàng.\nLúc gần 10h trưa nay (31/7), giá vàng giao ngay tại thị trường châu Á giảm 2 USD/oz so với đóng cửa phiên cuối tuần trước tại Mỹ, giao dịch ở mức 1.958,4 USD/oz. Mức giá này tương đương khoảng 56,3 triệu đồng/lượng nếu được quy đổi theo tỷ giá USD bán ra tại ngân hàng Vietcombank.\nCùng thời điểm, Công ty Bảo Tín Minh Châu báo giá vàng miếng SJC ở mức 66,58 triệu đồng/lượng (mua vào) và 67,18 triệu đồng/lượng (bán ra), giảm tương ứng 70.000 đồng/lượng và 50.000 đồng/lượng so với sáng thứ Bảy.\nNhẫn tròn trơn 999,9 hiệu Rồng Thăng Long của Bảo Tín Minh Châu có giá 56,23 triệu đồng/lượng và 57,08 triệu đồng/lượng, khôn

In [7]:
import pickle

with open("./data/data_tokened.pickle", "wb") as f:
  pickle.dump(data_token, f)

In [10]:
with open("./data/data_tokened.pickle", "rb") as f:
  data_token = pickle.load(f)
print(data_token[:5])

[['số_liệu', 'kinh_tế', 'mỹ', 'sẽ', 'tác_động', 'mạnh', 'tới', 'giá', 'vàng', '?', 'giá', 'vàng', 'thế_giới', 'khởi_động', 'tuần', 'giao_dịch', 'mới', 'trong', 'xu_thế', 'giảm', 'nhẹ', 'kéo', 'giá', 'vàng', 'trong', 'nước', 'giảm', 'theo', '_giới', 'phân_tích', 'cho', 'rằng', 'loạt', 'số_liệu', 'quan_trọng', 'của', 'kinh_tế', 'mỹ', 'công_bố', 'trong', 'tuần', 'này', 'có_thể', 'ảnh_hưởng', 'nhiều', 'đến', 'triển_vọng', 'lãi_suất', 'của', 'cục', 'dự_trữ', 'liên_bang', 'mỹ', 'fed', 'từ', 'đó', 'tác_động', 'không', 'nhỏ', 'đến', 'diễn_biến', 'giá', 'vàng', 'lúc', 'gần', 'h', 'trưa', 'nay', 'giá', 'vàng', 'giao', 'ngay', 'tại', 'thị_trường', 'châu_á', 'giảm', 'usd', 'oz', 'so', 'với', 'đóng', 'cửa', 'phiên', 'cuối', 'tuần', 'trước', 'tại', 'mỹ', 'giao_dịch', 'ở', 'mức', 'usd', 'oz', 'mức', 'giá', 'này', 'tương_đương', 'khoảng', 'triệu', 'đồng', 'lượng', 'nếu', 'được', 'quy_đổi', 'theo', 'tỷ_giá', 'usd', 'bán', 'ra', 'tại', 'ngân_hàng', 'vietcombank', 'cùng', 'thời_điểm', 'công_ty', 'bảo', '

In [13]:
from gensim.models import Word2Vec
from gensim.models import FastText

n = 50
ft_model = FastText(data_token, window=8, min_count=5, workers=2,sg=1)
ft_model.save('./data/fast_new.model')

In [14]:
ft_model.vector_size

100

In [16]:
print('model.wv["covid"]', ft_model.wv['covid'])

model.wv["covid"] [-0.04637375  0.50500125 -0.08610528 -0.06636415  0.08235595 -0.2315347
 -0.2189451  -0.17912175  0.0038097  -0.23709106 -0.36064395 -0.35797742
  0.01713946  0.2834053  -0.10240682 -0.10109159 -0.08093055 -0.48912412
  0.39023864 -0.25825226  0.2824941   0.00809003 -0.38856125 -0.19468261
 -0.3497471  -0.7638289  -0.21486335 -0.5167607  -0.33507454 -0.18829803
  0.34858912  0.40019816  0.40787724 -0.11996699  0.24949385  0.3638551
  0.25560543  0.17832832 -0.21318665  0.19424857  0.39179364 -0.33964917
  0.691486   -0.04285347 -0.02954821 -0.4791733  -0.03336978  0.00291532
 -0.0128537   0.4665111   0.20864347 -0.54186904  0.5047913   0.1129931
 -0.23053882  0.28673506  0.19436067  0.47040966  0.16268323  0.0846287
  0.05664866 -0.4636507   0.44213092 -0.6517372  -0.183071    0.6546702
  0.08126562 -0.07557086 -0.4070406   0.340135    0.03647118 -0.30854627
 -0.18071356  0.1648      0.00563814  0.53133506 -0.19440123  0.18305707
 -0.2668718  -0.08895018  0.19605206  

In [18]:
question = 'Các ngành nghề kinh doanh bất động sản'

question_tokens = clear_text_vi([question])
print(question_tokens)

# Word embeddings
question_embeddings = [ft_model.wv[word] for word in question_tokens[0] if word in ft_model.wv]

# Post embeddings
post_check = data_token[:40]

post_embeddings = []
for post in post_check:
    post_embedding = [ft_model.wv[word] for word in post if word in ft_model.wv]
    post_embeddings.append(post_embedding)

[['các', 'ngành_nghề', 'kinh_doanh', 'bất_động_sản']]


In [19]:
print('question_embeddings', question_embeddings)
print('post_embeddings[0]', post_embeddings[0])

question_embeddings [array([-2.34661147e-01,  1.45332098e-01, -2.24740639e-01,  1.12630673e-01,
       -3.55627209e-01, -5.48937954e-02,  1.76128298e-01,  4.50892121e-01,
        3.84262623e-03, -1.43959776e-01,  2.09090814e-01,  1.14934788e-04,
        2.37239808e-01,  3.41404527e-01, -2.43623152e-01,  1.80278987e-01,
        1.77633181e-01,  1.23508133e-01, -1.28631279e-01, -1.50760844e-01,
        1.57930642e-01, -6.95480853e-02, -2.56063133e-01, -2.80170944e-02,
       -1.94216177e-01, -5.60820103e-02,  4.07533854e-01, -3.76196474e-01,
       -4.40780163e-01, -5.03014848e-02,  1.68484136e-01, -1.38605282e-01,
       -3.06133330e-01,  2.95307469e-02,  6.29297048e-02, -2.22348019e-01,
        2.29145184e-01, -7.52151757e-02, -1.75245598e-01,  8.99023563e-02,
        1.65192947e-01, -2.02306226e-01,  2.89722443e-01,  1.60416111e-01,
        1.54488906e-01, -9.71181467e-02, -3.60922366e-01,  8.79151225e-02,
       -5.48133515e-02,  2.90591627e-01,  2.90335238e-01,  2.28051022e-01,
    

In [22]:
# Calculate sentence embeddings by averaging word embeddings
def calculate_sentence_embedding(embeddings):
    if len(embeddings) == 0:
        return np.zeros(model.vector_size)
    return np.mean(embeddings, axis=0)

question_embedding = calculate_sentence_embedding(question_embeddings)

mean_post_embedding = []
for post_embedding in post_embeddings:
    mean_post_embedding.append(calculate_sentence_embedding(post_embedding))

# Calculate similarity (cosine similarity)
from sklearn.metrics.pairwise import cosine_similarity
similarity_score = cosine_similarity([question_embedding], mean_post_embedding)

# Print the similarity score
print(f"Similarity Score: {similarity_score}")

Similarity Score: [[0.6514489  0.7387264  0.6453601  0.5927773  0.7662516  0.7880047
  0.6420051  0.6818201  0.60336864 0.6995888  0.64248455 0.6346873
  0.6516182  0.59185374 0.6638886  0.7003164  0.65813303 0.6555985
  0.7604061  0.6681079  0.59928256 0.7037468  0.71965635 0.75553024
  0.77339965 0.6772209  0.7364286  0.68661875 0.6305969  0.58182937
  0.731781   0.80420554 0.60331243 0.62650657 0.6959344  0.67808115
  0.6813012  0.5979552  0.6840123  0.5939404 ]]


In [23]:
# Convert the list of lists into a numpy array
np_similarity_score = np.array(similarity_score)
print('np_question_embedding', np_similarity_score)
# Sort the array in ascending order
sorted_indices = np.argsort(np_similarity_score[0])[::-1]

# Print the sorted array
print("sorted_indices", sorted_indices)

# Get the top 5 indices
top_5_indices = sorted_indices[:5]

# Print the sorted indices and the top 5 indices
print("Sorted Indices:", sorted_indices)
print("Top 5 Indices:", top_5_indices)

np_question_embedding [[0.6514489  0.7387264  0.6453601  0.5927773  0.7662516  0.7880047
  0.6420051  0.6818201  0.60336864 0.6995888  0.64248455 0.6346873
  0.6516182  0.59185374 0.6638886  0.7003164  0.65813303 0.6555985
  0.7604061  0.6681079  0.59928256 0.7037468  0.71965635 0.75553024
  0.77339965 0.6772209  0.7364286  0.68661875 0.6305969  0.58182937
  0.731781   0.80420554 0.60331243 0.62650657 0.6959344  0.67808115
  0.6813012  0.5979552  0.6840123  0.5939404 ]]
sorted_indices [31  5 24  4 18 23  1 26 30 22 21 15  9 34 27 38  7 36 35 25 19 14 16 17
 12  0  2 10  6 11 28 33  8 32 20 37 39  3 13 29]
Sorted Indices: [31  5 24  4 18 23  1 26 30 22 21 15  9 34 27 38  7 36 35 25 19 14 16 17
 12  0  2 10  6 11 28 33  8 32 20 37 39  3 13 29]
Top 5 Indices: [31  5 24  4 18]


In [24]:
for i, indice in enumerate(top_5_indices):
    # Remove duplicate spaces
    cleaned_sentence = ' '.join(data_list[indice][:200].split())
    print("Bài viết thứ " + str(i) + ":", cleaned_sentence)

Bài viết thứ 0: Đón đọc Tạp chí Kinh tế Việt Nam số 30-2023 Tạp chí Kinh tế Việt Nam số 30-2023 Để nâng cao chất lượng và phát triển thị trường trái phiếu doanh nghiệp riêng lẻ, cần ưu tiên phát triển các thị t
Bài viết thứ 1: Đón đọc Tạp chí Kinh tế Việt Nam số 31-2023 Tạp chí Kinh tế Việt Nam số 31-2023. Đã có một số tín hiệu tích cực từ sự khởi sắc của một số ngành và lĩnh vực trong nền kinh tế, nhưng doanh nghiệp
Bài viết thứ 2: Trung Quốc dè dặt phát tín hiệu kích cầu, chú trọng cứu bất động sản Các nhà lãnh đạo cấp cao nhất của Trung Quốc ngày 24/7 cam kết tăng cường hỗ trợ chính sách cho nền kinh tế trong bối cảnh sự phục
Bài viết thứ 3: Trung Quốc giảm đầu tư vào các nước phương Tây Mới chỉ vài năm trước, dòng vốn từ Trung Quốc còn đổ mạnh vào các nước giàu. Các nhà đầu tư “dày ví” của Trung Quốc đã chiếm nhiều dòng tít báo bằng nhữn
Bài viết thứ 4: Kinh tế Mỹ khó giảm phụ thuộc vào Trung Quốc? Theo tờ báo này, việc tách khỏi vai trò của Trung Quốc trong chuỗi cung ứng toàn cầu